# Air Demo Notebook

## Air One

### Extract the raw DataFrame

The project-level [extract](https://oiflib.readthedocs.io/en/latest/api.html#module-oiflib.extract) function extracts a DataFrame from an Excel or OpenDocument workbook based on the metadata contained in the [datasets.py](https://prose.io/#Defra-Data-Science-Centre-of-Excellence/oiflib/edit/initial/src/oiflib/datasets.py) dictionary.

For example, the dictionary contains the following metadata for Air One:

```
oif_datasets: Dict[str, Dict[str, Dict[str, Union[str, int]]]] = {
    "air": {
        "one": {
            "io": "http://uk-air.defra.gov.uk/reports/cat09/2010220959_DA_API_1990-2018_V1.0.xlsx",
            "sheet_name": "England API",
            "usecols": "B:AA",
            "skiprows": 13,
            "nrows": 1602,
        },
...
}
```

To import the `extract` function run:

In [3]:
from oiflib.extract import extract

To use this function, pass it the `theme` and `indicator` of the DataFrame you want to extract as lower-case strings: 

In [5]:
air_one_extracted = extract(theme="air", indicator="one")
air_one_extracted.head()

,ShortPollName,NFRCode,SourceName,1990,1995,1998,1999,2000,2001,2002,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,CO,1A1a,Autogenerators,0.000000,0.000000,0.000861,0.000987,0.001225,0.001165,0.001197,...,0.072842,0.188518,0.380950,0.773427,1.108654,1.654888,2.648169,4.194255,4.936837,5.536873
1,CO,1A1a,Miscellaneous industrial/commercial combustion,0.372920,0.285545,0.104246,0.108912,0.118016,0.123270,0.126718,...,0.135080,0.127430,0.144871,0.148156,0.172320,0.125973,0.133799,0.127731,0.144326,0.136516
2,CO,1A1a,Power stations,91.485663,85.075695,48.035092,43.356297,52.875139,53.176906,52.253263,...,49.778303,54.550899,56.543332,67.445893,65.244769,55.655886,46.738127,30.714808,30.172769,38.881950
3,CO,1A1a,Public sector combustion,0.009175,0.015521,0.014372,0.014410,0.012844,0.013138,0.014227,...,0.013591,0.015413,0.017175,0.017028,0.018264,0.018127,0.019443,0.019173,0.022132,0.021411
4,CO,1A1b,Refineries - combustion,4.455526,4.998781,5.248910,4.862012,3.785558,2.391528,2.788338,...,4.460696,3.287260,2.801700,3.162767,3.266939,4.154463,5.523977,4.606405,4.050079,3.934826


The [Air One module](https://oiflib.readthedocs.io/en/latest/api.html#air-one) contains functions to transform, enrich, and validate the extracted DataFrame.

To input these functions run:

In [7]:
from oiflib.air.one.enrich import enrich_air_one
from oiflib.air.one.transform import transform_air_one
from oiflib.air.one.validate import validate_air_one_extracted, validate_air_one_transformed, validate_air_one_enriched

### Validate the extracted DataFrame

`oiflib` uses the [pandera](https://pandera.readthedocs.io/en/stable/) package for schema-based data validation. The schemas for the extracted, transformed, and enriched Air One DataFrames are defined [here](https://prose.io/#Defra-Data-Science-Centre-of-Excellence/oiflib/edit/initial/src/oiflib/air/one/schemas.py). They define the column names, data-types, and allow you to [check](https://pandera.readthedocs.io/en/stable/checks.html) the values against various constraints.

For example, the schema of the extracted Air One DataFrame contains the following information:

```
schema_extracted: DataFrameSchema = DataFrameSchema(
    columns={
        "ShortPollName": Column(
            pandas_dtype=String,
            checks=[Check.isin(values_ShortPollName_extracted)],
        ),
        "NFRCode": Column(
            pandas_dtype=String,
            checks=[Check.isin(values_NFRCode)],
            nullable=True,
        ),
        "SourceName": Column(
            pandas_dtype=String,
            checks=[Check.isin(values_SourceName)],
            nullable=True,
        ),
        r"\d{4}": Column(pandas_dtype=Float, nullable=True, regex=True),
    },
    coerce=True,
    strict=True,
)
```
This schema checks for the existance of a "ShortPollName", a "NFRCode", and a "SourceName" column and any number of column names consisting of four digits. It checks that the first three contain string values, while the others contain float values. It also checks that the first three contain values from pre-defined lists.

Schemas such as this prower the validation functions. If the DataFrame passed to the validation function conforms to the schema, it is returned, if not the validation function raises an error.

For example, calling [validate_air_one_extracted](https://oiflib.readthedocs.io/en/latest/api.html#oiflib.air.one.validate.validate_air_one_extracted) on the DataFrame previously returned by `extract(theme="air", indicator="one")` returns that DataFrame.

In [ ]:
air_one_extracted_validated = validate_air_one_extracted(air_one_extracted)
air_one_extracted_validated.head()

However, calling it on another DataFrame results in a SchemaError.

In [8]:
extract(theme="air", indicator="two").pipe(validate_air_one_extracted)

SchemaError: column 'NCFormat' not in DataFrameSchema {'ShortPollName': <Schema Column: 'ShortPollName' type=str>, 'NFRCode': <Schema Column: 'NFRCode' type=str>, 'SourceName': <Schema Column: 'SourceName' type=str>, '\\d{4}': <Schema Column: '\d{4}' type=float>}

### Transform the extracted DataFrame

The [transform_air_one](https://oiflib.readthedocs.io/en/latest/api.html#oiflib.air.one.transform.transform_air_one) function combines the following actions:

- It applies a filter to "ShortPollName" to return only the total rows for the five pollutants.  
- It drops the unused "NFRCode" and "SourceName" columns.
- It removes " Total" from the "ShortPollName" column and changes “VOC” to “NMVOC”
- It unpivots the <year> column names into an EmissionYear column and their values into an Emission column.

In [10]:
air_one_transformed = transform_air_one(air_one_extracted_validated)
air_one_transformed.head()

,ShortPollName,EmissionYear,Emission
0,NOx,1990,2397.847344
1,PM2.5,1990,174.144903
2,SO2,1990,3134.835121
3,NMVOC,1990,2109.138181
4,NH3,1990,232.760876


Like `validate_air_one_extracted`, the [validate_air_one_transformed](https://oiflib.readthedocs.io/en/latest/api.html#oiflib.air.one.validate.validate_air_one_transformed) function checks the output against a schema.

In [11]:
air_one_transformed_validated = validate_air_one_transformed(air_one_transformed)
air_one_transformed_validated.head()

,ShortPollName,EmissionYear,Emission
0,NOx,1990,2397.847344
1,PM2.5,1990,174.144903
2,SO2,1990,3134.835121
3,NMVOC,1990,2109.138181
4,NH3,1990,232.760876


### Enrich the transformed DataFrame

Finally, the [enrich_air_one](https://oiflib.readthedocs.io/en/latest/api.html#oiflib.air.one.validate.validate_air_one_transformed) function adds an “Index” column with “Emission” values indexed to base year for each pollutant.

In [12]:
air_one_enriched = enrich_air_one(air_one_transformed_validated)
air_one_enriched.head()

,ShortPollName,EmissionYear,Emission,Index
0,NOx,1990,2397.847344,100.0
1,PM2.5,1990,174.144903,100.0
2,SO2,1990,3134.835121,100.0
3,NMVOC,1990,2109.138181,100.0
4,NH3,1990,232.760876,100.0


and the [validate_air_one_enriched](https://oiflib.readthedocs.io/en/latest/api.html#oiflib.air.one.validate.validate_air_one_enriched) function checks the output against a schema.

In [13]:
air_one_enriched_validated = validate_air_one_enriched(air_one_enriched)
air_one_enriched_validated

,ShortPollName,EmissionYear,Emission,Index
0,NOx,1990,2397.847344,100.000000
1,PM2.5,1990,174.144903,100.000000
2,SO2,1990,3134.835121,100.000000
3,NMVOC,1990,2109.138181,100.000000
4,NH3,1990,232.760876,100.000000
...,...,...,...,...
110,NOx,2018,604.799111,25.222586
111,PM2.5,2018,83.142806,47.743462
112,SO2,2018,118.684955,3.786003
113,NMVOC,2018,526.173425,24.947319


## Air Two

In [14]:
from oiflib.air.two.enrich import enrich_air_two
from oiflib.air.two.transform import transform_air_two
from oiflib.air.two.validate import validate_air_two_extracted, validate_air_two_transformed, validate_air_two_enriched

In [1]:
air_two_extracted = extract(theme="air", indicator="two").pipe(validate_air_two_extracted)
air_two_extracted.head()

NameError: name 'extract' is not defined

In [10]:
air_two_transformed = transform_air_two(air_two_extracted).pipe(validate_air_two_transformed)
air_two_transformed.head()

,NCFormat,IPCC,EmissionYear,CO2 Equiv
0,Agriculture,1A4ci_Agriculture/Forestry/Fishing:Stationary,1990,448.236698
1,Agriculture,1A4cii_Agriculture/Forestry/Fishing:Off-road,1990,2719.368825
2,Agriculture,2D1_Lubricant_Use,1990,7.767285
3,Agriculture,3A1a Enteric Fermentation - dairy cows,1990,4937.546160
4,Agriculture,3A1b Enteric Fermentation - other cattle,1990,6909.494541


In [12]:
air_two_enriched = enrich_air_two(air_two_transformed).pipe(validate_air_two_enriched)
air_two_enriched

,OIF_category,EmissionYear,CO2 Equiv
0,Agriculture,1990,33483.078737
1,Agriculture,1995,32023.748698
2,Agriculture,1998,31845.280086
3,Agriculture,1999,31873.301211
4,Agriculture,2000,30221.892227


## Air Three

In [6]:
from oiflib.air.three.extract import extract_air_three
from oiflib.air.three.transform import transform_air_three
from oiflib.air.three.validate import validate_air_three_extracted, validate_air_three_transformed

In [7]:
air_three_extracted = extract_air_three().pipe(validate_air_three_extracted)
air_three_extracted.head()

,Area code,PM2.5 2011 (total),PM2.5 2011 (non-anthropogenic),PM2.5 2011 (anthropogenic),Country,PM2.5 2012 (total),PM2.5 2012 (non-anthropogenic),PM2.5 2012 (anthropogenic),PM2.5 2013 (total),PM2.5 2013 (non-anthropogenic),...,PM2.5 2016 (anthropogenic),PM2.5 2017 (total),PM2.5 2017 (non-anthropogenic),PM2.5 2017 (anthropogenic),PM2.5 2018 (total),PM2.5 2018 (non-anthropogenic),PM2.5 2018 (anthropogenic),PM2.5 2019 (total),PM2.5 2019 (non-anthropogenic),PM2.5 2019 (anthropogenic)
0,Eng,12.0891,2.6357,9.4535,England,11.1081,2.1095,8.9986,11.5227,2.1314,...,9.4566,9.4113,0.5073,8.9040,9.5246,0.4429,9.0817,9.5741,0.5311,9.0430
1,Wal,9.4192,2.7214,6.6978,Wales,8.7860,2.1979,6.5880,9.9034,2.2408,...,7.5940,7.0468,0.5102,6.5366,7.7765,0.5999,7.1766,7.5190,0.5881,6.9308
2,Sco,8.0037,2.5759,5.4277,Scotland,7.6107,2.0355,5.5752,7.8215,2.0360,...,5.2794,5.8877,0.4607,5.4270,5.7886,0.3816,5.4070,5.5153,0.3569,5.1584
3,Nir,7.7701,2.7103,5.0598,Northern Ireland,7.6487,2.1100,5.5386,8.7661,2.2440,...,6.1638,6.1617,0.5045,5.6572,7.4441,0.5812,6.8629,5.8943,0.4429,5.4514


In [8]:
air_three_transformed = transform_air_three(air_three_extracted).pipe(validate_air_three_transformed)
air_three_transformed.head()

,Area code,Country,year,measure,ugm-3
0,Eng,England,2011,total,12.0891
12,Eng,England,2012,total,11.1081
24,Eng,England,2013,total,11.5227
36,Eng,England,2014,total,11.1680
48,Eng,England,2015,total,9.4215
